<a href="https://colab.research.google.com/github/dhtmddms/google_drive/blob/main/machine_learning_book/ch05_2_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

# 교차 검증이란
- 보통은 train set으로 모델을 훈련, test set으로 모델을 검증
- 이 과정을 반복하다보면, test set에 과적합하게 되므로, 실제 데이터로 예측을 수행하면 결과가 엉망으로 나올 수 있다
- 따라 이를 해결하고자 교차 검증을 수행
  + 보통 훈련 세트 60%, 검증 세트 20%, 테스트 세트 20% 로 나눔

- 장점
  + 정확도 향상, 데이터부족으로 인한 과소적합을 방지
  + 데이터 편중을 억제, 일반화된 모델 
- 단점
  + iteration 횟수가 많아 시간이 오래 걸림

출처:https://wooono.tistory.com/105

# 데이터 준비

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol','sugar','pH']]
target = wine ['class'].to_numpy()

# 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42
)

- 다시 훈련세트(sub)와 검증세트(val)로 분리

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state = 42
)
print(sub_input.shape, val_input.shape)


(4157, 3) (1040, 3)


# 결정 트리 모델 훈련

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


- 훈련 세트에 과대적합되어 있음을 확인

# 교차검증 수행
- 3-폴드 교차 검증
  + 훈련세트를 세 부분으로 나누어 교차검증을 수행하는 방식

In [ ]:
# 사이킷런의 교차 검증 함수 cross_validate()
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01388383, 0.01265836, 0.01324296, 0.01289964, 0.01294136]), 'score_time': array([0.00361514, 0.00347805, 0.00329566, 0.00336719, 0.00348496]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- 최종 점수 평균 구하기

In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


- cross_validate()는 훈련 데이터를 섞지 않고 폴드를 나눴음
- splitter()로 섞어준 후, 10-폴드 교차검증

In [ ]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle = True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼 파라미터 튜닝
- 사용자가 지정해야만 하는 파라미터 = 하이퍼 파라미터
- 머신러닝 라이브러리에서 클래스나 매개변수로 표현됨
- 라이브러리가 제공하는 기본값을 그대로사용하는 것을 추천

- 사이킷런의 GrodSearchCV 클래스는 하이퍼 파라미터 탐색과 교차검증을 한 번에 수행

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
    }
# dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42),params,n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련 -> best_estimator_

In [ ]:
dt = gs.best_estimator_
print(dt)
print(dt.score(train_input, train_target))

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
0.9615162593804117


- 그리드 서치로 찾은 최적의 매개변수는 best_params_에 저장되어 있음

In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


- 0.0001 이 가장 좋은 값으로 선택됨

In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


# 랜덤 서치
- 너무 많은 매개변수 조건이 있거나, 매개변수 값의 범위나 간격을 미리 정하기 어려울 때 유용한 방법
- 매개변수를 샘플링할 수 있는 확률분포 객체를 전달

In [ ]:
# uniform을 실숫값을 뽑고 randint는 정수값을 뽑는 클래스
from scipy.stats import uniform, randint

rgen = randint(0,10)
rgen.rvs(10)

array([5, 7, 7, 2, 4, 6, 9, 5, 6, 7])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96, 118, 107, 100,  80,  96, 111,  95,  95, 102]))

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.21258054, 0.69498992, 0.80667633, 0.43501987, 0.90512715,
       0.83735911, 0.26769318, 0.48205842, 0.90089121, 0.16716067])

# 그리드 서치 vs 랜덤 서치 
- 그리드 서치
  + 하이퍼 파라미터에 넣을 수 있는 값들을 순차적으로 입력한 뒤에 가장 높은 성능을 보이는 하이퍼 파라미터 값을 찾는 방법
  + 모델을 학생에 비유, 모의고사를 몇 번 칠지, 몇 번씩 개념을 볼 것인지 등등
  + 세부적인 사항들을 일일히 다 적용해봐가면서 어떤 규율이 가장 이 공부법(모델)에 적합한지 판단하는 것
  + 단점은 시간이 아주 많이 걸린다는 것

- 랜덤 서치
  + 하이퍼 파라미터 값을 랜덤하게 넣어보고 그중 우수한 값을 보인 하이퍼 파라미터를 활용해 모델을 생성
  + 장점은 불필요한 탐색횟수를 줄인다는 것

출처:https://huidea.tistory.com/32